In [259]:
# Recommendations
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import copy

import requests
from bs4 import BeautifulSoup as bs
import re

#### Steps
1. Clean Data, prepare for vectorization
    - Use themes, genres, titles, directors, cast, description? past ratings?
    - remove common words from descriptions
    - find a way to weight genres/themes, title more than description and directors
    - combine features into one word, so they are counted similarly
    - combine all used columns into one column
2. Pass Data to Scikit CountVectorizer
3. Use cosine similarity to generate similarities for all movies
4. Change columns and row indices to titles to correspond with similarity values
5. Use df.nlargest() to find the top 10 movies most similar to an input movie

In [260]:
lb_df = pd.read_csv("../Letterboxd_data/Letterboxd.csv")
lb_df = lb_df.rename(columns = {"Unnamed: 0": "Title"})
lb_df.head(4)

,Title,Year,Director,Desc.,Runtime,Cast,Watched,Lists,Liked,0.5,...,3,3.5,4,4.5,5,Average Rating,Total Rated,Genres,Themes,Premiere Date
0,Barbie,2023.0,Greta Gerwig,Barbie and Ken are having the time of their li...,114.0,"Margot Robbie, Ryan Gosling, America Ferrera, ...",3M,390K,1.3M,9788.0,...,288810.0,325457.0,682566.0,316056.0,690969.0,4.011,2486939.0,"Comedy, Fantasy, Adventure","Relationship comedy, Song and dance, Crude hum...",09 Jul 2023
1,Parasite,2019.0,Bong Joon-ho,"All unemployed, Ki-taek's family takes peculia...",133.0,"Song Kang-ho, Lee Sun-kyun, Cho Yeo-jeong, Cho...",3.4M,486K,2M,2270.0,...,80610.0,94836.0,508691.0,430357.0,1393803.0,4.555,2542023.0,"Comedy, Thriller, Drama","Intense violence and sexual transgression, Mov...",21 May 2019
2,Everything Everywhere All at Once,2022.0,Daniel Scheinert,An aging Chinese immigrant is swept up in an i...,140.0,"Michelle Yeoh, Stephanie Hsu, Ke Huy Quan, Jam...",2.6M,456K,1.3M,8470.0,...,121375.0,129423.0,389416.0,322424.0,1008809.0,4.363,2077462.0,"Science Fiction, Action, Adventure","Relationship comedy, Epic heroes, Intense comb...",11 Mar 2022
3,Fight Club,1999.0,David Fincher,A ticking-time-bomb insomniac and a slippery s...,139.0,"Edward Norton, Brad Pitt, Helena Bonham Carter...",3.3M,379K,1.5M,3096.0,...,151007.0,161694.0,621373.0,364705.0,865776.0,4.306,2230031.0,Drama,"Intense violence and sexual transgression, Pol...",10 Sep 1999


In [261]:
lb_df = lb_df.dropna()
lb_df.shape


(1911, 24)

#### Convert columns to int

In [262]:
lb_df[["0.5","1", "1.5", "2", "2.5", "3", "3.5", "4", "4.5", "5", "Runtime", "Year", "Total Rated"]] \
    = lb_df[["0.5","1", "1.5", "2", "2.5", "3", "3.5", "4", "4.5", "5", "Runtime", "Year", "Total Rated"]].astype(int)
lb_df.dtypes

Title              object
Year                int64
Director           object
Desc.              object
Runtime             int64
Cast               object
Watched            object
Lists              object
Liked              object
0.5                 int64
1                   int64
1.5                 int64
2                   int64
2.5                 int64
3                   int64
3.5                 int64
4                   int64
4.5                 int64
5                   int64
Average Rating    float64
Total Rated         int64
Genres             object
Themes             object
Premiere Date      object
dtype: object

In [263]:
def num_conversion(num):
    if num.strip()[-1].lower() == "m":
        return int(float(num.strip()[:-1]) * 1e6)  # Millions
    elif num.strip()[-1].lower() == "k":
        return int(float(num.strip()[:-1]) * 1e3)  # Thousands
    else:
        return int(num.strip()[:-1])  # Other

lb_df["Watched"] = lb_df["Watched"].apply(num_conversion)
lb_df["Lists"] = lb_df["Lists"].apply(num_conversion)
lb_df["Liked"] = lb_df["Liked"].apply(num_conversion)
lb_df.head()

,Title,Year,Director,Desc.,Runtime,Cast,Watched,Lists,Liked,0.5,...,3,3.5,4,4.5,5,Average Rating,Total Rated,Genres,Themes,Premiere Date
0,Barbie,2023,Greta Gerwig,Barbie and Ken are having the time of their li...,114,"Margot Robbie, Ryan Gosling, America Ferrera, ...",3000000,390000,1300000,9788,...,288810,325457,682566,316056,690969,4.011,2486939,"Comedy, Fantasy, Adventure","Relationship comedy, Song and dance, Crude hum...",09 Jul 2023
1,Parasite,2019,Bong Joon-ho,"All unemployed, Ki-taek's family takes peculia...",133,"Song Kang-ho, Lee Sun-kyun, Cho Yeo-jeong, Cho...",3400000,486000,2000000,2270,...,80610,94836,508691,430357,1393803,4.555,2542023,"Comedy, Thriller, Drama","Intense violence and sexual transgression, Mov...",21 May 2019
2,Everything Everywhere All at Once,2022,Daniel Scheinert,An aging Chinese immigrant is swept up in an i...,140,"Michelle Yeoh, Stephanie Hsu, Ke Huy Quan, Jam...",2600000,456000,1300000,8470,...,121375,129423,389416,322424,1008809,4.363,2077462,"Science Fiction, Action, Adventure","Relationship comedy, Epic heroes, Intense comb...",11 Mar 2022
3,Fight Club,1999,David Fincher,A ticking-time-bomb insomniac and a slippery s...,139,"Edward Norton, Brad Pitt, Helena Bonham Carter...",3300000,379000,1500000,3096,...,151007,161694,621373,364705,865776,4.306,2230031,Drama,"Intense violence and sexual transgression, Pol...",10 Sep 1999
4,Interstellar,2014,Christopher Nolan,The adventures of a group of explorers who mak...,169,"Matthew McConaughey, Anne Hathaway, Jessica Ch...",3100000,398000,1400000,3755,...,143190,139999,458472,307422,974730,4.360,2101383,"Science Fiction, Drama, Adventure","Monsters, aliens, sci-fi and the apocalypse, I...",26 Oct 2014


In [264]:
test = copy.copy(lb_df)


#### Remove all commas and punctuation in the needed columns

In [265]:
test["genres_new"] = test["Genres"].apply(str.replace, args=(" ", "")).apply(str.replace, args =(",", " ")).apply(str.lower)
test["cast_new"] = test["Cast"].apply(str.replace, args=(" ", "")).apply(str.replace, args =(",", " ")).apply(str.lower)
test["themes_new"] = test["Themes"].apply(str.replace, args=(" ", "")).apply(str.replace, args =(",", " ")).apply(str.lower)
punc = [",", ".", ";", ":", "!", "?", "/", "\"", "\'"]
for i in punc:
    test["Desc."] = test["Desc."].apply(str.replace, args=(i, ""))
test["Desc."] = test["Desc."].apply(str.replace, args = (i, "")).apply(str.lower)
test["director_new"] = test["Director"].apply(str.replace, args =(" ", "")).apply(str.lower)
test["title_new"] = test["Title"].apply(str.lower)
test.head()

,Title,Year,Director,Desc.,Runtime,Cast,Watched,Lists,Liked,0.5,...,Average Rating,Total Rated,Genres,Themes,Premiere Date,genres_new,cast_new,themes_new,director_new,title_new
0,Barbie,2023,Greta Gerwig,barbie and ken are having the time of their li...,114,"Margot Robbie, Ryan Gosling, America Ferrera, ...",3000000,390000,1300000,9788,...,4.011,2486939,"Comedy, Fantasy, Adventure","Relationship comedy, Song and dance, Crude hum...",09 Jul 2023,comedy fantasy adventure,margotrobbie ryangosling americaferrera katemc...,relationshipcomedy songanddance crudehumorands...,gretagerwig,barbie
1,Parasite,2019,Bong Joon-ho,all unemployed ki-taeks family takes peculiar ...,133,"Song Kang-ho, Lee Sun-kyun, Cho Yeo-jeong, Cho...",3400000,486000,2000000,2270,...,4.555,2542023,"Comedy, Thriller, Drama","Intense violence and sexual transgression, Mov...",21 May 2019,comedy thriller drama,songkang-ho leesun-kyun choyeo-jeong choiwoo-s...,intenseviolenceandsexualtransgression movingre...,bongjoon-ho,parasite
2,Everything Everywhere All at Once,2022,Daniel Scheinert,an aging chinese immigrant is swept up in an i...,140,"Michelle Yeoh, Stephanie Hsu, Ke Huy Quan, Jam...",2600000,456000,1300000,8470,...,4.363,2077462,"Science Fiction, Action, Adventure","Relationship comedy, Epic heroes, Intense comb...",11 Mar 2022,sciencefiction action adventure,michelleyeoh stephaniehsu kehuyquan jameshong ...,relationshipcomedy epicheroes intensecombatand...,danielscheinert,everything everywhere all at once
3,Fight Club,1999,David Fincher,a ticking-time-bomb insomniac and a slippery s...,139,"Edward Norton, Brad Pitt, Helena Bonham Carter...",3300000,379000,1500000,3096,...,4.306,2230031,Drama,"Intense violence and sexual transgression, Pol...",10 Sep 1999,drama,edwardnorton bradpitt helenabonhamcarter meatl...,intenseviolenceandsexualtransgression politics...,davidfincher,fight club
4,Interstellar,2014,Christopher Nolan,the adventures of a group of explorers who mak...,169,"Matthew McConaughey, Anne Hathaway, Jessica Ch...",3100000,398000,1400000,3755,...,4.360,2101383,"Science Fiction, Drama, Adventure","Monsters, aliens, sci-fi and the apocalypse, I...",26 Oct 2014,sciencefiction drama adventure,matthewmcconaughey annehathaway jessicachastai...,monsters aliens sci-fiandtheapocalypse imagina...,christophernolan,interstellar


In [266]:
# Remove most common words from descriptions, first must find what words are most common
value_counts = {}
for d in test["Desc."]:
    for word in d.split(" "):
        value_counts[word] = 1 + value_counts.get(word, 0)

value_counts = sorted(value_counts.items(), key= lambda k: k[1], reverse = True)

df = pd.DataFrame(value_counts)


In [267]:
# drop words that occur more than 30 times, because they likely won't provide any helpful grouping information
drop_words_list = list(df[df[1] > 30][0])
drop_words_list


['the',
 'a',
 'to',
 'and',
 'of',
 'in',
 'his',
 'is',
 'with',
 'her',
 'an',
 'he',
 'for',
 'their',
 'on',
 'that',
 'when',
 'as',
 'by',
 'from',
 'who',
 'but',
 'they',
 'into',
 'are',
 'life',
 'at',
 'has',
 'she',
 'after',
 'new',
 'him',
 'world',
 'young',
 'one',
 'up',
 'be',
 'out',
 'must',
 'family',
 'two',
 'it',
 'all',
 'will',
 'them',
 'find',
 'man',
 'love',
 'have',
 'only',
 'finds',
 'years',
 'help',
 'while',
 'where',
 'this',
 'story',
 'home',
 'about',
 'mysterious',
 'friends',
 'its',
 'school',
 'time',
 'not',
 'against',
 'woman',
 'lives',
 'back',
 'get',
 'through',
 'first',
 'soon',
 'can',
 'war',
 'own',
 'more',
 'himself',
 'becomes',
 'what',
 'now',
 'way',
 'most',
 'father',
 'which',
 '–',
 'over',
 'girl',
 'before',
 'city',
 'other',
 'was',
 'discovers',
 'between',
 'set',
 'son',
 'been',
 'during',
 'take',
 'off',
 'begins',
 'wife',
 'down',
 'together',
 'group',
 'three',
 'named',
 'takes',
 'high',
 'past',
 'town'

In [268]:
def remove_words(input_string, words_to_remove):
    # Split the input string into words
    words = input_string.split()
    # Remove specified words
    filtered_words = [word for word in words if word not in words_to_remove]
    # Join the filtered words back into a string
    return ' '.join(filtered_words)

desc_list = []
for d in test["Desc."]:
    desc_list.append(remove_words(d, drop_words_list))

test["desc_new"] = desc_list
test.head()

,Title,Year,Director,Desc.,Runtime,Cast,Watched,Lists,Liked,0.5,...,Total Rated,Genres,Themes,Premiere Date,genres_new,cast_new,themes_new,director_new,title_new,desc_new
0,Barbie,2023,Greta Gerwig,barbie and ken are having the time of their li...,114,"Margot Robbie, Ryan Gosling, America Ferrera, ...",3000000,390000,1300000,9788,...,2486939,"Comedy, Fantasy, Adventure","Relationship comedy, Song and dance, Crude hum...",09 Jul 2023,comedy fantasy adventure,margotrobbie ryangosling americaferrera katemc...,relationshipcomedy songanddance crudehumorands...,gretagerwig,barbie,barbie ken having colorful seemingly perfect b...
1,Parasite,2019,Bong Joon-ho,all unemployed ki-taeks family takes peculiar ...,133,"Song Kang-ho, Lee Sun-kyun, Cho Yeo-jeong, Cho...",3400000,486000,2000000,2270,...,2542023,"Comedy, Thriller, Drama","Intense violence and sexual transgression, Mov...",21 May 2019,comedy thriller drama,songkang-ho leesun-kyun choyeo-jeong choiwoo-s...,intenseviolenceandsexualtransgression movingre...,bongjoon-ho,parasite,unemployed ki-taeks peculiar interest wealthy ...
2,Everything Everywhere All at Once,2022,Daniel Scheinert,an aging chinese immigrant is swept up in an i...,140,"Michelle Yeoh, Stephanie Hsu, Ke Huy Quan, Jam...",2600000,456000,1300000,8470,...,2077462,"Science Fiction, Action, Adventure","Relationship comedy, Epic heroes, Intense comb...",11 Mar 2022,sciencefiction action adventure,michelleyeoh stephaniehsu kehuyquan jameshong ...,relationshipcomedy epicheroes intensecombatand...,danielscheinert,everything everywhere all at once,aging chinese immigrant swept insane alone wha...
3,Fight Club,1999,David Fincher,a ticking-time-bomb insomniac and a slippery s...,139,"Edward Norton, Brad Pitt, Helena Bonham Carter...",3300000,379000,1500000,3096,...,2230031,Drama,"Intense violence and sexual transgression, Pol...",10 Sep 1999,drama,edwardnorton bradpitt helenabonhamcarter meatl...,intenseviolenceandsexualtransgression politics...,davidfincher,fight club,ticking-time-bomb insomniac slippery soap sale...
4,Interstellar,2014,Christopher Nolan,the adventures of a group of explorers who mak...,169,"Matthew McConaughey, Anne Hathaway, Jessica Ch...",3100000,398000,1400000,3755,...,2101383,"Science Fiction, Drama, Adventure","Monsters, aliens, sci-fi and the apocalypse, I...",26 Oct 2014,sciencefiction drama adventure,matthewmcconaughey annehathaway jessicachastai...,monsters aliens sci-fiandtheapocalypse imagina...,christophernolan,interstellar,adventures explorers use newly discovered worm...


#### To Do:
- Add all words to one column
- Scale more important columns by determined factors
    - Columns to scale: Genre, Theme, Director?, Title?

In [269]:
test_final = test[["title_new", "genres_new", "themes_new", "cast_new", "director_new", "desc_new"]]
test_final.head()

,title_new,genres_new,themes_new,cast_new,director_new,desc_new
0,barbie,comedy fantasy adventure,relationshipcomedy songanddance crudehumorands...,margotrobbie ryangosling americaferrera katemc...,gretagerwig,barbie ken having colorful seemingly perfect b...
1,parasite,comedy thriller drama,intenseviolenceandsexualtransgression movingre...,songkang-ho leesun-kyun choyeo-jeong choiwoo-s...,bongjoon-ho,unemployed ki-taeks peculiar interest wealthy ...
2,everything everywhere all at once,sciencefiction action adventure,relationshipcomedy epicheroes intensecombatand...,michelleyeoh stephaniehsu kehuyquan jameshong ...,danielscheinert,aging chinese immigrant swept insane alone wha...
3,fight club,drama,intenseviolenceandsexualtransgression politics...,edwardnorton bradpitt helenabonhamcarter meatl...,davidfincher,ticking-time-bomb insomniac slippery soap sale...
4,interstellar,sciencefiction drama adventure,monsters aliens sci-fiandtheapocalypse imagina...,matthewmcconaughey annehathaway jessicachastai...,christophernolan,adventures explorers use newly discovered worm...


In [270]:
# Scaling
test_final["genres_new"] = test_final["genres_new"].apply(lambda x: (" " + str(x)) * 2).apply(str.strip)
test_final["themes_new"] = test_final["themes_new"].apply(lambda x: (" " + str(x) * 2)).apply(str.strip)
test_final["director_new"] = test_final["director_new"].apply(lambda x: (" " + str(x)) * 3).apply(str.strip)

test_final.head()


/var/folders/1m/lbc4h2hx1k967vn2sytxsb480000gn/T/ipykernel_24196/918191749.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_final["genres_new"] = test_final["genres_new"].apply(lambda x: (" " + str(x)) * 2).apply(str.strip)
/var/folders/1m/lbc4h2hx1k967vn2sytxsb480000gn/T/ipykernel_24196/918191749.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_final["themes_new"] = test_final["themes_new"].apply(lambda x: (" " + str(x) * 2)).apply(str.strip)
/var/folders/1m/lbc4h2hx1k967vn2sytxsb480000gn/T

,title_new,genres_new,themes_new,cast_new,director_new,desc_new
0,barbie,comedy fantasy adventure comedy fantasy adventure,relationshipcomedy songanddance crudehumorands...,margotrobbie ryangosling americaferrera katemc...,gretagerwig gretagerwig gretagerwig,barbie ken having colorful seemingly perfect b...
1,parasite,comedy thriller drama comedy thriller drama,intenseviolenceandsexualtransgression movingre...,songkang-ho leesun-kyun choyeo-jeong choiwoo-s...,bongjoon-ho bongjoon-ho bongjoon-ho,unemployed ki-taeks peculiar interest wealthy ...
2,everything everywhere all at once,sciencefiction action adventure sciencefiction...,relationshipcomedy epicheroes intensecombatand...,michelleyeoh stephaniehsu kehuyquan jameshong ...,danielscheinert danielscheinert danielscheinert,aging chinese immigrant swept insane alone wha...
3,fight club,drama drama,intenseviolenceandsexualtransgression politics...,edwardnorton bradpitt helenabonhamcarter meatl...,davidfincher davidfincher davidfincher,ticking-time-bomb insomniac slippery soap sale...
4,interstellar,sciencefiction drama adventure sciencefiction ...,monsters aliens sci-fiandtheapocalypse imagina...,matthewmcconaughey annehathaway jessicachastai...,christophernolan christophernolan christophern...,adventures explorers use newly discovered worm...


#### Join all the columns into one column/series of the DataFrame

In [281]:
test_final['data'] = test_final[test_final.columns].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)

print(test_final['data'].head())

0    barbie comedy fantasy adventure comedy fantasy...
1    parasite comedy thriller drama comedy thriller...
2    everything everywhere all at once scienceficti...
3    fight club drama drama intenseviolenceandsexua...
4    interstellar sciencefiction drama adventure sc...
Name: data, dtype: object


/var/folders/1m/lbc4h2hx1k967vn2sytxsb480000gn/T/ipykernel_24196/4015124862.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_final['data'] = test_final[test_final.columns].apply(


#### Pass this series to a Count Vectorizer and get a similarity matrix from cosine_similarity

In [282]:
vectorizer = CountVectorizer()

vectorized = vectorizer.fit_transform(test_final['data'])

In [273]:
similarities = cosine_similarity(vectorized)
print(similarities)

[[1.         0.04451319 0.10780183 ... 0.04194292 0.04194292 0.        ]
 [0.04451319 1.         0.02266007 ... 0.09918507 0.         0.06488857]
 [0.10780183 0.02266007 1.         ... 0.         0.19216473 0.        ]
 ...
 [0.04194292 0.09918507 0.         ... 1.         0.         0.09171268]
 [0.04194292 0.         0.19216473 ... 0.         1.         0.        ]
 [0.         0.06488857 0.         ... 0.09171268 0.         1.        ]]


In [274]:
sim_df = pd.DataFrame(similarities, columns = test_final["title_new"], index = test_final["title_new"]).reset_index()
sim_df.head()

title_new,title_new,barbie,parasite,everything everywhere all at once,fight club,interstellar,joker,spider-man: into the spider-verse,knives out,la la land,...,black bear,the house bunny,meet the fockers,god’s own country,dolemite is my name,letters to juliet,big eyes,bringing out the dead,hulk,the vanishing
0,barbie,1.000000,0.044513,0.107802,0.022257,0.043171,0.000000,0.056860,0.065407,0.125073,...,0.022495,0.358746,0.297805,0.000000,0.287472,0.191204,0.026307,0.041943,0.041943,0.000000
1,parasite,0.044513,1.000000,0.022660,0.052632,0.040835,0.098433,0.000000,0.041246,0.086041,...,0.095750,0.035720,0.040242,0.153897,0.090640,0.127667,0.149302,0.099185,0.000000,0.064889
2,everything everywhere all at once,0.107802,0.022660,1.000000,0.000000,0.219767,0.011772,0.173672,0.011099,0.000000,...,0.000000,0.057671,0.086630,0.013804,0.073171,0.011451,0.026784,0.000000,0.192165,0.000000
3,fight club,0.022257,0.052632,0.000000,1.000000,0.163342,0.229676,0.000000,0.010311,0.043021,...,0.074472,0.017860,0.020121,0.051299,0.045320,0.063833,0.049767,0.079348,0.099185,0.021630
4,interstellar,0.043171,0.040835,0.219767,0.163342,1.000000,0.042429,0.104324,0.000000,0.041723,...,0.041272,0.000000,0.000000,0.049752,0.043953,0.041272,0.048266,0.038478,0.442492,0.000000


In [285]:
# Function(s) to recommend movies based on a string input of movie titles

def filter_top(movie):
    return movie in list(sim_df.title_new)

# input movies as a string separated by comma-spaces

def recommend(input_movies):
    n = 5
    input_movies = input_movies.lower().split(", ")
    input_movies = pd.Series(input_movies)
    input_movies = input_movies[input_movies.apply(filter_top)]
    recs = pd.DataFrame()
    recs_list = []
    for i, movie in enumerate(input_movies):  
        try:
            recs = pd.concat([recs, pd.DataFrame(sim_df.nlargest(n,movie)[['title_new', movie]])])
            recs_list.extend([s for s in recs[movie]][(i * n):(i+1)*n + 1])
        except KeyError:
            print("Please enter a valid movie")
    recs = recs.reset_index().rename(columns = {"title_new": "recs"})
    recs["sim"] = recs_list
    recs = recs.sort_values(by="sim", ascending=False).drop_duplicates("recs")
    recs = recs[~recs["recs"].isin(input_movies)]
    
    return recs.recs[:10]

#### Next:
- Take username(s) and scrape their top/most recent movies
- Use this input for recommendations

(Using my username as an example/test)


In [286]:

username = "ejlarsen"
resp = requests.get(f"https://letterboxd.com/{username}/films/diary/")
html_text = bs(resp.text, "html.parser")

movie_list = [tag.text for tag in html_text.findAll("h3")]
recommend(", ".join(movie_list))

191               anchorman 2: the legend continues
21     harry potter and the deathly hallows: part 1
111                    star wars: the force awakens
112       star wars: episode i – the phantom menace
113                         the empire strikes back
114                star wars: the rise of skywalker
162        harry potter and the philosopher’s stone
116                  jurassic world: fallen kingdom
61                                      glass onion
163         harry potter and the chamber of secrets
Name: recs, dtype: object

In [292]:
def username_recommend(usernames):

    combined_movies = []
    for u in usernames.strip().split(", "):
        resp = requests.get(f"https://letterboxd.com/{u}/films/diary/")
        html_text = bs(resp.text, "html.parser")

        movie_list = [tag.text for tag in html_text.findAll("h3")]
        
        combined_movies.extend(movie_list)

    combined_movies = list(set(combined_movies))

    return recommend(", ".join(combined_movies))

username_recommend("ejlarsen, thefieldraccoon")

96                                  the conjuring 2
246                                  it chapter two
97                                        insidious
86                anchorman 2: the legend continues
131                                           ted 2
216    harry potter and the deathly hallows: part 1
98                             insidious: chapter 2
236                    star wars: the force awakens
237       star wars: episode i – the phantom menace
238                         the empire strikes back
Name: recs, dtype: object